In [12]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import sqlite3
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()


In [2]:
nba_teams = teams.get_teams()
nba_teams_id = {}
for x in nba_teams:
    name = x['full_name']
    nba_teams_id[f'{name} Id'] = x['id']
ids = nba_teams_id.values()


In [3]:
from nba_api.stats.endpoints import leaguegamefinder
games_frame = leaguegamefinder.LeagueGameFinder(date_from_nullable="2024-10-22",
                                                date_to_nullable="2025-4-13", 
                                                season_type_nullable="Regular Season",
                                                league_id_nullable='00')
# games = [team for team in games_frame if team['TEAM_ID'] in nba_teams]
games = games_frame.get_data_frames()[0]
games = pd.DataFrame(games)
games.columns


/Users/rohilverma/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [4]:
##Offensive Team Statistics
def EFG(x):
    num = float(x['FGM']) + 0.5 * float(x['FG3M'])
    denom = float(x['FGA'])
    return round(num/denom, 3)
def TOVP(x):
    denom = float(x['FGA']) + 0.44 * x['FTA'] + float(x['TOV'])
    return round(float(x['TOV']) / denom, 4)
games['TDREB'] = games.groupby('GAME_ID')['DREB'].transform('sum')
def OREBP(x):
    x['OPPDREB'] = x['TDREB'] - x['DREB']
    return float(x['OREB']) / (float(x['OREB'] + x['OPPDREB']))
def FTRATE(x):
    return float(x['FTA']) / float(x['FGA'])

In [5]:
##Defensive Team Statistics
games["TotalFGMade"] = games.groupby('GAME_ID')['FGM'].transform('sum')
games["TotalFGAtt"] = games.groupby('GAME_ID')['FGA'].transform('sum')
games['Total3PM'] = games.groupby('GAME_ID')['FG3M'].transform('sum')
def OppEFG(x):
    num = float(x["TotalFGMade"] - x["FGM"]) + 0.5 * float(x["Total3PM"] - x["FG3M"])
    denom = float(x['TotalFGAtt'] - x["FGA"])
    return round(num/denom, 3)
games["TotalTOV"] = games.groupby('GAME_ID')['TOV'].transform('sum')
games["TotalFTA"] = games.groupby('GAME_ID')['FTA'].transform('sum')
def OppTOVP(x):
    num = float(x['TotalTOV'] - x['TOV'])
    denom = float(x["TotalFGAtt"] - x["FGA"]) + 0.44 * float(x["TotalFTA"] - x["FTA"]) + num
    return round(num/denom, 3)
games['TotalOREB'] = games.groupby('GAME_ID')['OREB'].transform('sum')
games['OppOREB'] = games['TotalOREB'] - games['OREB']
def DREBP(x):
    return float(x['DREB']) / float(x['DREB'] + x['OppOREB'])
def OppFTRATE(x):
    return float(x['TotalFTA'] - x['FTA']) / float(x['TotalFGAtt'] - x['FGA'])


In [6]:
games['EFG'] = games.apply(EFG, axis=1)
games['TOVP'] = games.apply(TOVP, axis=1)
games['OREBP'] = games.apply(OREBP, axis=1)
games['FTRATE'] = games.apply(FTRATE, axis=1)
games['OppEFG'] = games.apply(OppEFG, axis=1)
games['OppTOVP'] = games.apply(OppTOVP, axis=1)
games['DREBP'] = games.apply(DREBP, axis=1)
games['OppFTRATE'] = games.apply(OppFTRATE, axis=1)


In [ ]:
games.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'])

In [8]:
games.shape

(2460, 44)

In [11]:
games.to_sql("2024-2025 Team Data",conn,if_exists="replace",index=False)

2460